# Dataset Modeling
We have 2 different data sources:
1) csv files containing song information (lyrics-data.csv)
2) txt files for each artist, containing the lyrics of their songs

The goal is to model those sources into a single dataset for further analysis and preprocessing steps.

In [310]:
import pandas as pd
import os

## Prepare each dataframe - one from csv, other from txt

### CSV file - lyrics-data.csv

In [311]:
df_from_csv = pd.read_csv("../data/raw/csv/lyrics-data.csv")

In [312]:
df_from_csv.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [313]:
df_from_csv.tail()

,ALink,SName,SLink,Lyric,language
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en
379930,/clegg-johnny/,Woman Be My Country,/clegg-johnny/woman-be-my-country.html,Here we stand on the edge of the day\nFaces me...,en


In [314]:
df_from_csv.columns.array

<NumpyExtensionArray>
['ALink', 'SName', 'SLink', 'Lyric', 'language']
Length: 5, dtype: object

The useful columns are 'Lyric' which obviously contains the lyrics of a song and 'ALink' which represents the song's author. The author will be used to avoid duplicate lyrics which may come from the text files.

But before, since we need only english songs, let's check the values of the 'language' column.

In [315]:
df_from_csv["language"].value_counts()

language
en     191814
pt     157393
es       9917
rw       1679
it       1432
fr       1225
de        844
fi        145
sv        112
ro         97
no         89
is         86
tl         69
pl         47
gl         36
ga         32
tr         32
id         26
cy         23
su         19
af         19
sw         19
ko         17
nl         14
da         13
ca         13
et         13
ms          8
ja          7
st          7
ht          5
gd          4
ru          4
ar          4
eu          4
cs          3
ku          3
ny          3
mg          3
lg          2
jw          2
hu          2
vi          1
iw          1
sr          1
hmn         1
hr          1
fa          1
sq          1
zh          1
lv          1
sl          1
Name: count, dtype: int64

In [316]:
df_from_csv = df_from_csv[df_from_csv["language"] == "en"]

In [317]:
df_from_csv = df_from_csv.drop(["SName", "SLink", "language"], axis=1)

In [318]:
df_from_csv.head()

,ALink,Lyric
69,/ivete-sangalo/,I feel so unsure\nAs I take your hand and lead...
86,/ivete-sangalo/,"Don't let them fool, ya\nOr even try to school..."
88,/ivete-sangalo/,"Baby, let's cruise, away from here\nDon't be c..."
111,/ivete-sangalo/,"Know it sounds funny\nBut, I just can't stand ..."
140,/ivete-sangalo/,You've got that look again\nThe one I hoped I ...


Checking data types.

In [319]:
df_from_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191814 entries, 69 to 379930
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ALink   191814 non-null  object
 1   Lyric   191814 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


Handling missing data:
1) There are less non-null object (string) in column 'Lyric' than there are entries.
2) There is one less object (string) in column 'ALink' than there are entries.
3) They could possibly be filled with information from the text files data, which means we can remove the NaN values here. Joining with information from the text files will add the originaly missing information, if any exists though.

In [320]:
df_from_csv = df_from_csv.dropna()

We would like that artist names are formated in a way which makes merging with the text files easier. The convention would be [lowercase]-[lowercase].

In [321]:
df_from_csv['ALink'].values

array(['/ivete-sangalo/', '/ivete-sangalo/', '/ivete-sangalo/', ...,
       '/clegg-johnny/', '/clegg-johnny/', '/clegg-johnny/'], dtype=object)

In [322]:
df_from_csv['ALink'] = df_from_csv['ALink'].apply(lambda x: x.replace("/", ""))

In [323]:
df_from_csv['ALink'].values

array(['ivete-sangalo', 'ivete-sangalo', 'ivete-sangalo', ...,
       'clegg-johnny', 'clegg-johnny', 'clegg-johnny'], dtype=object)

### Text files

Let's create a data frame from the text files. The column 'ALink' which represents artist names will be constructed from the file names. The column 'Lyric' will have the contents of those file for each artis.

In [324]:
files = os.listdir("../data/raw/txt/")
for file in files:
    if file != ".gitkeep":
        print(file)

adele.txt
al-green.txt
alicia-keys.txt
amy-winehouse.txt
beatles.txt
bieber.txt
bjork.txt
blink-182.txt
bob-dylan.txt
bob-marley.txt
britney-spears.txt
bruce-springsteen.txt
bruno-mars.txt
cake.txt
dickinson.txt
disney.txt
dj-khaled.txt
dolly-parton.txt
dr-seuss.txt
drake.txt
eminem.txt
janisjoplin.txt
jimi-hendrix.txt
johnny-cash.txt
joni-mitchell.txt
kanye-west.txt
kanye.txt
Kanye_West.txt
lady-gaga.txt
leonard-cohen.txt
lil-wayne.txt
Lil_Wayne.txt
lin-manuel-miranda.txt
lorde.txt
ludacris.txt
michael-jackson.txt
missy-elliott.txt
nickelback.txt
nicki-minaj.txt
nirvana.txt
notorious-big.txt
notorious_big.txt
nursery_rhymes.txt
patti-smith.txt
paul-simon.txt
prince.txt
r-kelly.txt
radiohead.txt
rihanna.txt


In [325]:
file_data = []
for file_name in os.listdir("../data/raw/txt/"):
    file_path = os.path.join("../data/raw/txt/", file_name)
    if os.path.isfile(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            file_content = file.read()
            if file_name[:-4] != ".git":
                file_data.append({"ALink": file_name[:-4], "Lyric": file_content})
df_from_txt = pd.DataFrame(file_data)

In [326]:
df_from_txt.head()

,ALink,Lyric
0,adele,Looking for some education\nMade my way into t...
1,al-green,"Let's stay together I, I'm I'm so in love with..."
2,alicia-keys,Ooh....... New York x2 Grew up in a town that ...
3,amy-winehouse,Build your dreams to the stars above\nBut when...
4,beatles,"Yesterday, all my troubles seemed so far away\..."


In [327]:
df_from_txt.tail()

,ALink,Lyric
44,paul-simon,"Hey, Vietnam, Vietnam, Vietnam, Vietnam\nVietn..."
45,prince,\n\nAll of this and more is for you\nWith love...
46,r-kelly,"I hear you callin', ""Here I come baby""\nTo sav..."
47,radiohead,"Come on, come on\nYou think you drive me crazy..."
48,rihanna,"Ghost in the mirror\nI knew your face once, bu..."


In [328]:
df_from_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ALink   49 non-null     object
 1   Lyric   49 non-null     object
dtypes: object(2)
memory usage: 916.0+ bytes


## Check for repeated information
Let's see if the data from the text files already has the information contained in the data from the csv file. In that case, we can drop the repeated data from the csv file dataframe. The best way to check this is through the artists.

In [329]:
df_from_csv["ALink"].value_counts()

ALink
frank-sinatra          819
elvis-presley          747
dolly-parton           723
matheus-hardke         707
lil-wayne              689
                      ... 
santanna-o-cantador      1
gatinha-manhosa          1
bia-socek                1
toni-tornado             1
gabriela-rocha           1
Name: count, Length: 2508, dtype: int64

In [330]:
df_from_txt["ALink"].value_counts()

ALink
adele                 1
kanye-west            1
Kanye_West            1
lady-gaga             1
leonard-cohen         1
lil-wayne             1
Lil_Wayne             1
lin-manuel-miranda    1
lorde                 1
ludacris              1
michael-jackson       1
missy-elliott         1
nickelback            1
nicki-minaj           1
nirvana               1
notorious-big         1
notorious_big         1
nursery_rhymes        1
patti-smith           1
paul-simon            1
prince                1
r-kelly               1
radiohead             1
kanye                 1
joni-mitchell         1
al-green              1
johnny-cash           1
alicia-keys           1
amy-winehouse         1
beatles               1
bieber                1
bjork                 1
blink-182             1
bob-dylan             1
bob-marley            1
britney-spears        1
bruce-springsteen     1
bruno-mars            1
cake                  1
dickinson             1
disney                1
dj-khaled 

I see some examples like "Kanye-West", "kanye-west" and "kanye". Let's use a name format [lowercase]-[lowercase] and then merge the rows with the same 'ALink' value.

In [331]:
def transform_artist_name(x):
    x=x.replace("_", "-")
    x=x.lower()
    return x

In [332]:
df_from_txt["ALink"] = df_from_txt["ALink"].apply(transform_artist_name)

In [333]:
df_from_txt["ALink"].value_counts()

ALink
kanye-west            2
lil-wayne             2
notorious-big         2
adele                 1
missy-elliott         1
kanye                 1
lady-gaga             1
leonard-cohen         1
lin-manuel-miranda    1
lorde                 1
ludacris              1
michael-jackson       1
nickelback            1
al-green              1
nicki-minaj           1
nirvana               1
nursery-rhymes        1
patti-smith           1
paul-simon            1
prince                1
r-kelly               1
radiohead             1
joni-mitchell         1
johnny-cash           1
jimi-hendrix          1
janisjoplin           1
alicia-keys           1
amy-winehouse         1
beatles               1
bieber                1
bjork                 1
blink-182             1
bob-dylan             1
bob-marley            1
britney-spears        1
bruce-springsteen     1
bruno-mars            1
cake                  1
dickinson             1
disney                1
dj-khaled             1
dolly-part

Let's join Kanye West's and Lil Wayne's lyrics into one dataframe row.

In [334]:
df_from_txt = df_from_txt.groupby("ALink")["Lyric"].agg(lambda x: '\n'.join(x)).reset_index()

In [335]:
df_from_txt["ALink"].value_counts()

ALink
adele                 1
missy-elliott         1
kanye                 1
kanye-west            1
lady-gaga             1
leonard-cohen         1
lil-wayne             1
lin-manuel-miranda    1
lorde                 1
ludacris              1
michael-jackson       1
nickelback            1
al-green              1
nicki-minaj           1
nirvana               1
notorious-big         1
nursery-rhymes        1
patti-smith           1
paul-simon            1
prince                1
r-kelly               1
radiohead             1
joni-mitchell         1
johnny-cash           1
jimi-hendrix          1
janisjoplin           1
alicia-keys           1
amy-winehouse         1
beatles               1
bieber                1
bjork                 1
blink-182             1
bob-dylan             1
bob-marley            1
britney-spears        1
bruce-springsteen     1
bruno-mars            1
cake                  1
dickinson             1
disney                1
dj-khaled             1
dolly-part

The only problem remaining now is "kanye" and "kanye-west". The solution should also check for these occurances with other artists.

Identify values where one is contained in the other.

In [336]:
to_merge = {}
for idx, row in df_from_txt.iterrows():
    for other_idx, other_row in df_from_txt.iterrows():
        if idx != other_idx and row["ALink"] in other_row["ALink"]:
            if other_idx not in to_merge:
                to_merge[other_idx] = []
            to_merge[other_idx].append(idx)

Merge values.

In [337]:
for other_idx, merge_indices in to_merge.items():
    merged_values = '\n'.join(df_from_txt.at[idx, "Lyric"] for idx in merge_indices)
    df_from_txt.at[other_idx, "Lyric"] += '\n' + merged_values

Drop the rows with duplicates and reset the index.

In [338]:
df_from_txt.drop(index=[idx for indices in to_merge.values() for idx in indices], inplace=True)
df_from_txt.reset_index(drop=True, inplace=True)

In [339]:
df_from_txt["ALink"].value_counts()

ALink
adele                 1
johnny-cash           1
kanye-west            1
lady-gaga             1
leonard-cohen         1
lil-wayne             1
lin-manuel-miranda    1
lorde                 1
ludacris              1
michael-jackson       1
missy-elliott         1
nickelback            1
nicki-minaj           1
nirvana               1
notorious-big         1
nursery-rhymes        1
patti-smith           1
paul-simon            1
prince                1
r-kelly               1
radiohead             1
joni-mitchell         1
jimi-hendrix          1
al-green              1
janisjoplin           1
alicia-keys           1
amy-winehouse         1
beatles               1
bieber                1
bjork                 1
blink-182             1
bob-dylan             1
bob-marley            1
britney-spears        1
bruce-springsteen     1
bruno-mars            1
cake                  1
dickinson             1
disney                1
dj-khaled             1
dolly-parton          1
dr-seuss  

Intersection of the "ALink" columns from both dataframes.

In [340]:
intersection = list(set(df_from_txt["ALink"]) & set(df_from_csv["ALink"]))
intersection

['bob-dylan',
 'nicki-minaj',
 'rihanna',
 'drake',
 'r-kelly',
 'nickelback',
 'dj-khaled',
 'lorde',
 'amy-winehouse',
 'kanye-west',
 'adele',
 'jimi-hendrix',
 'bruno-mars',
 'dolly-parton',
 'johnny-cash',
 'bjork',
 'missy-elliott',
 'al-green',
 'patti-smith',
 'blink-182',
 'alicia-keys',
 'michael-jackson',
 'prince',
 'bob-marley',
 'ludacris',
 'bruce-springsteen',
 'nirvana',
 'eminem',
 'disney',
 'radiohead',
 'lady-gaga',
 'joni-mitchell',
 'lil-wayne',
 'britney-spears',
 'leonard-cohen']

Number of common values from both dataframes.

In [341]:
len(intersection)

35

Number of unique values from the csv dataframe.

In [342]:
len(set(df_from_csv["ALink"]))

2508

Number of unique values from the txt dataframe.

In [343]:
len(set(df_from_txt["ALink"]))

45

Now we need to check the 'Lyric' values of the intersect elements from the csv dataframe and see if they appear in the 'Lyric' values of those intersect elements in the txt dataframe.

This is hard to do programaticaly because of possible various text formatings in the data sources. There are extensive possibilities to cover.

In [349]:
filtered_df = df_from_csv[df_from_csv["ALink"].isin(intersection)]
len(filtered_df)

9483

After visual inspection, a large number of "Lyric" values from the csv dataframe appear in the "Lyric" values from the txt dataframe but in different formats. The agreement is to keep the repeating information because in textual datasets words, sentences and even paragraphs can and most likely will repeat.

The dataframes can now be concatenated and saved as an interim dataset.